In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_excel("totalNPdata.xlsx",  header=None)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,26,66,180,64.00,125.75,16.25,1.13,0.90,653.33,...,25.5,1.21,0.95,670.0,390.0,105.0,1082.5,225.0,61.5,66.0
1,0,26,66,180,67.25,131.75,10.75,1.45,1.10,546.67,...,33.5,1.36,0.98,605.0,400.0,100.0,965.0,210.0,64.5,59.0
2,0,26,66,180,70.00,129.75,17.00,1.34,1.04,563.33,...,24.5,1.41,1.00,610.0,390.0,95.0,982.5,195.0,66.0,57.0
3,0,26,66,180,62.50,125.50,15.50,1.10,0.87,726.67,...,35.5,1.13,0.89,690.0,410.0,120.0,1132.5,250.0,79.0,80.5
4,0,26,66,180,61.75,122.50,13.50,0.94,0.76,830.00,...,25.5,0.98,0.79,805.0,470.0,150.0,1290.0,335.0,77.5,69.5


In [5]:
df.shape[0]

218

In [6]:
index = list(range(df.shape[0]))


In [7]:
import random
random.shuffle(index)
miss_index = index[:10]
attr = 6#步速
true_value = df.iloc[miss_index, attr]
true_value

128    10.00
129    14.50
53     10.25
134    13.25
68     10.00
180    20.50
150    25.50
165     9.25
71     16.00
125    11.00
Name: 6, dtype: float64

In [8]:
df.iloc[miss_index, attr] = np.nan

In [9]:
print(np.sum(df[attr]))
print(np.mean(df[attr]))

3513.2200000000003
16.89048076923077


In [10]:
#使用平均值填补缺失数据
mean_ = np.mean(df[attr])
df.iloc[miss_index, attr] = mean_

In [11]:
#计算平均相对误差
def print_error(value, true_value, method):
    error = true_value - value
    rel_error = np.abs(error / true_value)
    mean_rel_error = np.mean(rel_error)
    print("各缺失值的相对误差")
    print("序号\t相对误差")
    print(rel_error)
    print("使用%s填补缺失值的平均相对误差为%d%%" %(method, mean_rel_error*100))

In [12]:
print_error(mean_, true_value, "平均值法")

各缺失值的相对误差
序号	相对误差
128    0.689048
129    0.164861
53     0.647852
134    0.274753
68     0.689048
180    0.176074
150    0.337628
165    0.825998
71     0.055655
125    0.535498
Name: 6, dtype: float64
使用平均值法填补缺失值的平均相对误差为43%


In [13]:
#重置缺失值
df.iloc[miss_index, attr] = np.nan

In [14]:
df_miss = df.loc[miss_index]
df_mr = df.drop(miss_index, axis=0)
miss = df_miss 

In [15]:
miss_no_attr = df_miss.drop(attr, axis=1)
mr_no_attr = df_mr.drop(attr, axis=1)

In [16]:
knn_value = true_value.copy()
for idx in miss_index:
    d = mr_no_attr - miss_no_attr.loc[idx]
    d = np.square(d)
    d = np.sum(d, axis=1)


In [44]:
d1 = d.sort_values()
k = 4
d1 = d1[:k]
v = np.mean(df_mr.iloc[d1.index.tolist(), attr])
print_error(v, true_value, "K-邻近补缺算法")

各缺失值的相对误差
序号	相对误差
128    0.337500
129    0.077586
53     0.304878
134    0.009434
68     0.337500
180    0.347561
150    0.475490
165    0.445946
71     0.164062
125    0.215909
Name: 6, dtype: float64
使用K-邻近补缺算法填补缺失值的平均相对误差为27%


# 加权的K-邻近补缺算法暂时不写，下面先实现EM算法

可观测到的样本X={X1, X2，……, Xn}和不可观测到的样本Z={Z1, Z2，……, Zn}组成的，则Y = X∪Z。

In [18]:
X = df_miss
Z = df_mr
Y = df

In [94]:
from scipy.stats import multivariate_normal as mvn
def em_gmm(xs, pis, mus, sigmas, tol=0.01, max_iter=100):

    n, p = xs.shape
    k = len(pis)

    ll_old = 0
    for _ in range(max_iter):
        exp_A = []
        exp_B = []
        ll_new = 0

        # E-step
        ws = np.zeros((k, n))
        for j in range(k):
            for i in range(n):
                ws[j, i] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
        ws /= ws.sum(0)

        # M-step
        pis = np.zeros(k)
        for j in range(k):
            for i in range(n):
                pis[j] += ws[j, i]
        pis /= n

        mus = np.zeros((k, p))
        for j in range(k):
            for i in range(n):
                mus[j] += ws[j, i] * xs[i]
            mus[j] /= ws[j, :].sum()

        sigmas = np.zeros((k, p, p))
        for j in range(k):
            for i in range(n):
                ys = np.reshape(xs[i]- mus[j], (4,1))
                sigmas[j] += ws[j, i] * np.dot(ys, ys.T)
            sigmas[j] /= ws[j,:].sum()

        # update complete log likelihoood
        ll_new = 0.0
        for i in range(n):
            s = 0
            for j in range(k):
                s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
            ll_new += np.log(s)

        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new

    return ws, pis, mus, sigmas

## 选取2，3，4，5列进行聚类，主要是考虑到太多属性需要初始化的参数太多

In [49]:
idx_gmm = list(range(2,6))
df_gmm = df[idx_gmm]
df_gmm.shape

(218, 4)

### 共218个样本，4个特征，所以n=218，假设GMM由5个高斯分布混合成，则mu的维度应为(5,4)，sigma的维度应为(5,4),
### pi的维度应为(5,)

In [118]:
#Initial the parameters
n = 218
_shape = (5, df_gmm.shape[1])
_mus = np.random.random(_shape)
_sigmas = np.random.random(_shape)
_pis = np.random.random(_shape[0])
_pis = _pis / np.sum(_pis)
xs = np.array(df_gmm)

In [117]:
a = np.array(np.mean(df_gmm))

a1=np.stack((a,a,a,a,a), axis=0)
print(a)
print(a1)

[  59.41743119  165.16055046   59.76885321  119.17220183]
[[  59.41743119  165.16055046   59.76885321  119.17220183]
 [  59.41743119  165.16055046   59.76885321  119.17220183]
 [  59.41743119  165.16055046   59.76885321  119.17220183]
 [  59.41743119  165.16055046   59.76885321  119.17220183]
 [  59.41743119  165.16055046   59.76885321  119.17220183]]


In [119]:
ws, pis, mus, sigmas = em_gmm(np.array(df_gmm), _pis, _mus, _sigmas, tol=0.01, max_iter=100)


C:\Users\40712\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


ValueError: array must not contain infs or NaNs

In [61]:
np.array(df_gmm)

array([[  66.  ,  180.  ,   64.  ,  125.75],
       [  66.  ,  180.  ,   67.25,  131.75],
       [  66.  ,  180.  ,   70.  ,  129.75],
       [  66.  ,  180.  ,   62.5 ,  125.5 ],
       [  66.  ,  180.  ,   61.75,  122.5 ],
       [  66.  ,  180.  ,   60.5 ,  122.  ],
       [  53.  ,  172.  ,   68.  ,  132.5 ],
       [  53.  ,  172.  ,   71.  ,  143.  ],
       [  53.  ,  172.  ,   71.5 ,  146.  ],
       [  53.  ,  172.  ,   76.  ,  156.  ],
       [  53.  ,  172.  ,   77.5 ,  154.5 ],
       [  53.  ,  172.  ,   81.  ,  167.  ],
       [  50.  ,  163.  ,   66.  ,  132.  ],
       [  50.  ,  163.  ,   60.75,  118.25],
       [  50.  ,  163.  ,   61.5 ,  122.25],
       [  49.  ,  158.  ,   40.83,   79.5 ],
       [  49.  ,  158.  ,   50.25,  101.5 ],
       [  74.  ,  177.  ,   77.5 ,  156.  ],
       [  74.  ,  177.  ,   78.  ,  154.  ],
       [  51.  ,  157.  ,   58.5 ,  118.5 ],
       [  51.  ,  157.  ,   55.5 ,  111.75],
       [  51.  ,  157.  ,   57.  ,  117.  ],
       [  

In [27]:
xs[:10]

array([[ 1.67689766,  4.92395945],
       [-2.18947366,  4.40885556],
       [ 0.07317587,  3.04278779],
       [ 0.87739551,  4.33482605],
       [ 0.65223875,  5.36323088],
       [ 0.90642396,  3.01294716],
       [ 0.32507405,  4.95406967],
       [-1.04267383,  3.12154895],
       [ 2.75127973,  4.66708046],
       [ 1.46081383,  4.01597148]])